In [1]:
import pandas as pd

In [2]:
P_gtf = pd.read_csv('/Data_2/Daehwa/Data_Library/GTF_parsed/v0.7.1/gencode.vM27.annotation.gtf/Processed_gtf.tsv', sep='\t')
P_gtf = P_gtf[['Transcript_ID', 'Gene_ID', 'Gene_type', 'Gene_name', 'Strand', 'Seqname', 'CCDS_tag', 'Transcript_range', 'Transcript_length', '5UTR_length', 'CDS_length', '3UTR_length']]
P_gtf = P_gtf.rename(columns={'Transcript_ID':'transcript_id', 'Gene_ID':'gene_id', 'Gene_name':'gene_name'})

repre = pd.read_csv('/Data_2/Daehwa/Data_Library/representative-isoforms.txt', sep='\t', header=None, usecols=[0,1], names=['gene_id','transcript_id'])
repre_GTF = pd.merge(repre, P_gtf, on=['gene_id','transcript_id'])

# protein coding
repre_GTF = repre_GTF[ (repre_GTF['5UTR_length']+repre_GTF['CDS_length']+repre_GTF['3UTR_length'])!=0 ]

display(repre_GTF)

,gene_id,transcript_id,Gene_type,gene_name,Strand,Seqname,CCDS_tag,Transcript_range,Transcript_length,5UTR_length,CDS_length,3UTR_length
0,ENSMUSG00000000001.5,ENSMUST00000000001.5,protein_coding,Gnai3,-,chr3,Yes,"108014596,108053462",3262,141,1065,2056
1,ENSMUSG00000000003.16,ENSMUST00000000003.14,protein_coding,Pbsn,-,chrX,Yes,"76881507,76897229",902,140,525,237
2,ENSMUSG00000000028.16,ENSMUST00000000028.14,protein_coding,Cdc45,-,chr16,Yes,"18599197,18630722",2143,313,1701,129
4,ENSMUSG00000000037.18,ENSMUST00000019101.11,protein_coding,Scml2,+,chrX,Yes,"159945768,160041192",4842,501,2718,1623
5,ENSMUSG00000000049.12,ENSMUST00000000049.6,protein_coding,Apoh,+,chr11,Yes,"108286119,108305222",1190,51,1038,101
...,...,...,...,...,...,...,...,...,...,...,...,...
52349,ENSMUSG00000118670.1,ENSMUST00000239545.1,protein_coding,Muc19,+,chr15,No,"91722531,91832447",22800,41,22575,184
52350,ENSMUSG00000118671.1,ENSMUST00000239552.1,protein_coding,Eppk1,-,chr15,No,"75973330,76004395",12332,1133,10377,822
52351,ENSMUSG00000118672.1,ENSMUST00000239554.1,protein_coding,Muc4,+,chr16,No,"32555015,32602596",12361,183,11973,205
53206,ENSMUSG00001074846.1,ENSMUST00010126032.3,protein_coding,Iqcf3,-,chr9,Yes,"106420585,106438830",1131,59,612,460


In [3]:
# Target genes
genes = {'gene_name':['Tef']}
genes = pd.DataFrame(genes)

genes = pd.merge(genes, repre_GTF, on='gene_name').set_index('gene_name').T
display(genes)

gene_name,Tef
gene_id,ENSMUSG00000022389.16
transcript_id,ENSMUST00000109553.10
Gene_type,protein_coding
Strand,+
Seqname,chr15
CCDS_tag,Yes
Transcript_range,"81686622,81711064"
Transcript_length,4403
5UTR_length,363
CDS_length,858


In [4]:
# Select range for genes
margin = 0.05

ranges = {}
for gene in genes:

    start = round(int(genes[gene]['Transcript_range'].split(',')[0]) * (1-margin))
    end   = round(int(genes[gene]['Transcript_range'].split(',')[1]) * (1+margin))

    ranges[gene] = f"{genes[gene]['Seqname']}:{start}-{end}"

display(ranges)

{'Tef': 'chr15:77602291-85796617'}

In [72]:
# Counting coverage of reads
# Here I used 'samtools depth'. If you want to calculate the coverage of paired sequencing bam files, use 'samtools mpileup' to prevent overlapped count.
SPs = [f'D{day}{rep}' for day in ['0','4','8'] for rep in ['a','b','c']]

for gene in ranges:
    for SP in SPs:
        !samtools depth -r {ranges[gene]} /Data_2/Jun/Adipocytes/alignments/novaseq/{SP}.sorted.bam | gzip > {gene}_{SP}_RPF_depth.tsv.gz

In [5]:
# Select range for genes

ranges = {}
for gene in genes:
    
    ranges[gene] = f"{genes[gene]['transcript_id']}:1-{int(genes[gene]['Transcript_length'])}"

display(ranges)

{'Tef': 'ENSMUST00000109553.10:1-4403'}

In [6]:
# Counting coverage of reads
# Here I used 'samtools depth'. If you want to calculate the coverage of paired sequencing bam files, use 'samtools mpileup' to prevent overlapped count.
SPs = [f'D{day}{rep}' for day in ['0','4','8'] for rep in ['a','b','c']]

for gene in ranges:
    for SP in SPs:
        !samtools depth -r {ranges[gene]} /Data_2/Jun/Adipocytes/tr-aln/novaseq/{SP}.rep.bam | gzip > {gene}_{SP}_RPF_T_depth.tsv.gz